Notebook used to generate data for alldots.ipynb

In [1]:
import glob
import numpy as np
import itertools
import pandas as pd
import matplotlib.pyplot as plt
import BootstrapFunctions as bsf

In [2]:
standard_columns = ['E[rhoxy]','E[CVx/CVy]','E[<x>]','E[<y>]','E[<F_x>]','E[<F_y>]',
                    '97.5% (rhoxy)','2.5% (rhoxy)','97.5% (CVx/CVy)','2.5% (CVx/CVy)',
                    '97.5% (<x>)','2.5% (<x>)','97.5% (<y>)','2.5% (<y>)',
                    '97.5% (<F_x>)','2.5% (<F_x>)','97.5% (<F_y>)','2.5% (<F_y>)',
                    'F_y at avg','F_x at avg','Var[lambda]','Var[gamma]','E[lambda]','E[gamma]',
                    'consistent fy signs (avg)','consistent fy signs (at avg)',
                    'consistent fx signs (avg)','consistent fx signs (at avg)']
parentfolders = ['BothFb','BothFb_fasty','BothFb_hardcoded','BothFb_Nothardcoded','BothFb_Nothardcoded_allfb',
                 'BothFb_Nothardcoded_fxlt1test','Bothfb_Violator_aboveline_largeN_plusbonus',
                 'fx_violations_withpdist','NegFy1','NegFy2','NegFylt1','NoFb','PosFy1']
#parentfolders = ['NoFb']

In [3]:
paramlist = ['lambda','beta_x','gamma','beta_y','nx','Kx','ny','Ky','offset']
byfilelist = ['fileN','fileda','filedb','sysname']

In [4]:
#List out the perturbation types: u is up, d is down, n is none. l for lambda, g for gamma.  Add 1,2 to balance the neutral points
pertcases = {"lngn":[0,1,2],"lngu":[3,4,5],"lngd":[6,7,8],
             "lugn":[9,10,11],"lugu":[12,13,14],"lugd":[15,16,17],
             "ldgn":[18,19,20],"ldgu":[21,22,23],"ldgd":[24,25,26]}
oneparamperts = pertcases["lngn"]+pertcases["lugn"]+pertcases["ldgn"]+pertcases["lngu"]+pertcases["lngd"]
no_nonperts = pertcases["lugn"]+pertcases["ldgn"]+pertcases["lngu"]+pertcases["lngd"]

In [5]:
def rate(x,y,l,nx,Kx,ny,Ky,o):
    return l*((x**nx)/(Kx**nx+x**nx) + (y**ny)/(Ky**ny+y**ny) + o)
def rate_deriv_x(x,y,l,nx,Kx,ny,Ky,o):
    return l*nx*(x**(nx-1))*(Kx**nx)/(Kx**nx+x**nx)**2
def rate_deriv_y(x,y,l,nx,Kx,ny,Ky,o):
    return l*ny*(y**(ny-1))*(Ky**ny)/(Ky**ny+y**ny)**2
def sensitivty_x(x,y,l,nx,Kx,ny,Ky,o):
    return x*rate_deriv_x(x,y,l,nx,Kx,ny,Ky,o)/rate(x,y,l,nx,Kx,ny,Ky,o)
def sensitivty_y(x,y,l,nx,Kx,ny,Ky,o):
    return y*rate_deriv_y(x,y,l,nx,Kx,ny,Ky,o)/rate(x,y,l,nx,Kx,ny,Ky,o)

In [6]:
no_feedback_no_int = []
just_mrna_feedback = []
just_protein_feedback = []
both_feedback = []
names = ['2D_no_feedback_no_int','2D_just_mrna_feedback','2D_just_protein_feedback','2D_both_feedback']

In [10]:
#For sorting data to display parameters
for parent in parentfolders:
    folders = glob.glob(f'Data/{parent}/*/*')
    dat = []
    failedsims = []
    bootstrapdicts = []
    for i,folder in enumerate(folders):
        files = glob.glob(folder+'/*data.csv',recursive=True)
        for j,f in enumerate(files):
            data = pd.read_csv(f)
            data = data.reset_index().rename(columns={'index':'perturbation'})
            numstrings = f.split("_")[-2].split('n')
            if (data[['Cov Bal xx','Cov Bal yy','Cov Bal xy']] > 0.05).any(axis=1).any():
                failedsims.append(f)
                continue
            data['fileN'] = float(numstrings[0])
            data['fileda'] = float(numstrings[1])
            data['filedb'] = float(numstrings[2])
            data['sysname'] = "/".join(f.split("/")[:-1])
            if 'offset' not in data:
                data['offset'] = 0
            dat.append(data)
            bsdict = bsf.pertexp_interpret(data,'all')
            fys = sensitivty_y(data['<x>'].astype(float),data['<y>'].astype(float),
                                            data['lambda'],data['nx'].astype(float),data['Kx'],
                                            data['ny'].astype(float),data['Ky'],data['offset'])
            fxs = sensitivty_x(data['<x>'].astype(float),data['<y>'].astype(float),
                                            data['lambda'],data['nx'].astype(float),data['Kx'],
                                            data['ny'].astype(float),data['Ky'],data['offset'])
            bsdict['F_y at avg'] = fys.mean()
            bsdict['F_x at avg'] = fxs.mean()
            bsdict['consistent fy signs (avg)'] = np.sum(np.sign(data['<F_y>']))
            bsdict['consistent fx signs (avg)'] = np.sum(np.sign(1-data['<F_x>']))
            bsdict['consistent fy signs (at avg)'] = np.sum(np.sign(fys))
            bsdict['consistent fx signs (at avg)'] = np.sum(np.sign(1-fxs))
            bsdict['fileN'] = float(numstrings[0])
            bsdict['fileda'] = float(numstrings[1])
            bsdict['filedb'] = float(numstrings[2])
            bsdict['sysname'] = "/".join(f.split("/")[:-1])
            for p in paramlist:
                try:
                    bsdict[p] = data[p][0]
                except KeyError:
                    bsdict[p] = 0
            if data['nx'][0] == 0 and data['ny'][0] == 0:
                if bsdict['offset'] != 0:
                    bsdict['lambda'] *= bsdict['offset']
                    bsdict['offset'] = 0
                no_feedback_no_int.append(bsdict)
            elif data['nx'][0] != 0 and data['ny'][0] == 0:
                just_mrna_feedback.append(bsdict)
            elif data['nx'][0] == 0 and data['ny'][0] != 0:
                just_protein_feedback.append(bsdict)
            else:
                both_feedback.append(bsdict)
    print(f"Finished {parent}")
for i,dataset in enumerate([no_feedback_no_int,just_mrna_feedback,just_protein_feedback,both_feedback]):
    df = pd.DataFrame(dataset)
    df.set_index(['fileN','fileda','filedb','sysname'],inplace=True)
    df[paramlist+['E[<F_x>]','E[<F_y>]','F_y at avg','F_x at avg','consistent fy signs (avg)','consistent fx signs (avg)',
                  'consistent fy signs (at avg)','consistent fx signs (at avg)']].to_csv(f'Data/parameters/{names[i]}.csv')

Finished BothFb
Finished BothFb_fasty
Finished BothFb_hardcoded
Finished BothFb_Nothardcoded
Finished BothFb_Nothardcoded_allfb
Finished BothFb_Nothardcoded_fxlt1test
Finished Bothfb_Violator_aboveline_largeN_plusbonus
Finished fx_violations_withpdist
Finished NegFy1
Finished NegFy2
Finished NegFylt1
Finished NoFb
Finished PosFy1
